#**DALL-3**

by Philipuss#4066

Everything was taken from [this repo](https://github.com/Jack000/DALLE-pytorch) by Jack000.

In [ ]:
#@title <font color="lightgreen" size="+3">←</font> <font size="+2">🔆</font> **Set Up [RUN ME]** <font size="+2">🔆</font>

download_main_libraries = True #@param {type:"boolean"}
download_vqgan = True #@param {type:"boolean"}
download_dalle = True #@param {type:"boolean"}

if download_main_libraries:
  !git clone https://github.com/openai/CLIP                  &>/dev/null
  !pip install -e ./CLIP                                     &>/dev/null
  !git clone https://github.com/Jack000/DALLE-pytorch        &>/dev/null
  !pip install -e ./DALLE-pytorch                            &>/dev/null

if download_vqgan:
  !curl -L -o vqgan.yaml --http1.1 'https://heibox.uni-heidelberg.de/f/b24d14998a8d4f19a34f/?dl=1'  &> /dev/null
  !curl -L -o vqgan.pt   --http1.1 'https://heibox.uni-heidelberg.de/f/34a747d5765840b5a99d/?dl=1'  &> /dev/null
if download_dalle:
  !curl -OL --http1.1 'https://dall-3.com/models/dalle/bpe.model'       &> /dev/null
  !curl -OL --http1.1 'https://dall-3.com/models/dalle/dalle-latest.pt' &> /dev/null

from einops import repeat
from einops import rearrange
import math
import numpy as np
from PIL import Image
import sys

%cd /content/DALLE-pytorch
import torch
from torchvision.utils import make_grid, save_image
from torchvision.transforms import functional as TF
from dalle_pytorch import DiscreteVAE, OpenAIDiscreteVAE, VQGanVAE, DALLE
from dalle_pytorch.tokenizer import tokenizer, HugTokenizer, YttmTokenizer, ChineseTokenizer
%cd ../
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sys.path.append('./CLIP')
sys.path.append('./DALLE-pytorch')
tokenizer = YttmTokenizer('bpe.model')
load_obj = torch.load('dalle-latest.pt', map_location='cpu')
dalle_params, vae_params, weights = load_obj.pop('hparams'), load_obj.pop('vae_params'), load_obj.pop('weights')
dalle_params.pop('vae', None)
vae = VQGanVAE('vqgan.pt', 'vqgan.yaml')
dalle = DALLE(vae = vae, **dalle_params).to(device)
dalle.load_state_dict(weights)

print("Finished!")

In [ ]:
#@title <font color="lightgreen" size="+3">←</font> <font size="+2">🎇</font> **Run DALL-3** <font size="+2">🎇</font>

prompt = 'a photo of a chicken.' #@param {type:"string"}
randomness = 0.84 #@param {type:"number"}
top_p =  0.8#@param {type:"number"}
batch_size = 1 #@param {type:"number"}
batches = 1 #@param {type:"number"}

prompt = prompt.lower()
text_tokens = tokenizer.tokenize([prompt], dalle.text_seq_len).to(device)
text_tokens = repeat(text_tokens, '() n -> b n', b = batch_size)
outputs = []
image_tokens = []
for i in range(batches):
  out, tok = dalle.generate_images(text_tokens, temperature=randomness, top_p_thresh = top_p, return_tokens = True)  
  outputs.append(out)
  for j in range(batch_size):
    pimg = TF.to_pil_image(out[j])
    display(pimg)
    pimg.save('image' + str(i) + '.png')
    image_tokens.append(tok[j])